In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import gensim.downloader as api
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import contractions


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
df = pd.read_csv("Tweets.csv")
df = df[['airline_sentiment', 'text']]
df.columns = ['Sentiment', 'Tweet']


In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = contractions.fix(text)
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"@\w+", '', text)
    text = re.sub(r"#\w+", '', text)
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"\d+", '', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens


In [ ]:
df['Tokens'] = df['Tweet'].apply(preprocess)

In [ ]:
w2v_model = api.load('word2vec-google-news-300')

In [ ]:
def vectorize(tokens, model, dim=300):
    vectors = [model[word] for word in tokens if word in model]
    if len(vectors) == 0:
        return np.zeros(dim)
    return np.mean(vectors, axis=0)

df['Vector'] = df['Tokens'].apply(lambda x: vectorize(x, w2v_model))


In [ ]:
X = np.vstack(df['Vector'].values)
y = df['Sentiment'].map({'positive': 1, 'neutral': 0, 'negative': -1}).values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.7739071038251366


In [ ]:
def predict_tweet_sentiment(model, w2v_model, tweet):
    tokens = preprocess(tweet)
    vec = vectorize(tokens, w2v_model).reshape(1, -1)
    pred = model.predict(vec)[0]
    if pred == 1:
        return "positive"
    elif pred == 0:
        return "neutral"
    else:
        return "negative"


In [ ]:
print(predict_tweet_sentiment(clf, w2v_model, "I love the service on Delta Airlines!"))
print(predict_tweet_sentiment(clf, w2v_model, "Flight was okay, nothing special."))
print(predict_tweet_sentiment(clf, w2v_model, "Worst flight ever. Very disappointed!"))


positive
negative
negative
